In [1]:
#%pip install --extra-index-url https://test.pypi.org/simple ../../EnergyPlus-OOEP/
#%pip install git+https://github.com/NTU-CCA-HVAC-OPTIM-a842a748/EnergyPlus-Datasets.git
#%pip install ipywidgets tqdm gymnasium torch ray[rllib]

# 🤖 Reinforcement Learning

TODO description

Kickstart the simulation.

In [2]:
#import logging as _logging_
#_logging_.basicConfig(level='INFO')
#from energyplus.ooep.addons.logging import LogProvider

from energyplus.ooep.addons.progress import ProgressProvider

from energyplus.ooep import (
    Simulator,
    Model,
    Weather,
    Report,
)

from energyplus.dataset.basic import dataset as _epds_

simulator = Simulator().add(
    ProgressProvider(),
    #LogProvider(),
)

simulator.awaitable.run_forever(
    input=Simulator.InputSpecs(
        model=(
            _epds_.models / 'ASHRAE901_OfficeLarge_STD2019_Denver_Chiller205_Detailed.idf'
        ),
        weather=(_epds_.weathers / 'USA_CO_Denver-Aurora-Buckley.AFB.724695_TMY3.epw'),
    ),
    output=Simulator.OutputSpecs(
        #report=('/tmp/ooep-report-9e1287d2-8e75-4cf5-bbc5-f76580b56a69'),
    ),
    options=Simulator.RuntimeOptions(
        #design_day=True,
    ),
)

  0%|          | 0/100 [00:00<?, ?it/s]

<Task pending name='Task-5' coro=<Engine.run_forever() running at /home/user@AD/lab/EnergyPlus-OOEP/packages/energyplus/ooep/utils/awaitables.py:68>>

Configure the controller (an RLlib algorithm).

In [ ]:
import numpy as _numpy_
import gymnasium as _gymnasium_

from energyplus.ooep.addons.rl import (
    VariableBox,
    SimulatorEnv,
)

from energyplus.ooep import (
    Actuator,
    OutputVariable,
)

from ray.rllib.algorithms.ppo import PPOConfig



config = (
    PPOConfig.from_dict({        
        'create_env_on_local_worker': True
    })
    .resources(num_gpus=1)
    .environment(
        SimulatorEnv, 
        env_config=SimulatorEnv.Config(
            action_space=_gymnasium_.spaces.Dict({
                'thermostat': VariableBox(
                    low=15., high=20.,
                    dtype=_numpy_.float32,
                    shape=(),
                ).bind(Actuator.Ref(
                    type='Zone Temperature Control',
                    control_type='Heating Setpoint',
                    key='CORE_MID',
                ))
            }),    
            observation_space=_gymnasium_.spaces.Dict({
                'temperature': VariableBox(
                    low=-_numpy_.inf, high=+_numpy_.inf,
                    dtype=_numpy_.float32,
                    shape=(),
                ).bind(OutputVariable.Ref(
                    type='People Air Temperature',
                    key='CORE_MID',
                )),
            }),
            reward_function=lambda __obs: 1,
            event_refs=[
                'begin_zone_timestep_after_init_heat_balance',
            ],
            simulator_factory=lambda simulator=simulator: simulator,
        )
    )
    .rollouts(
        #num_rollout_workers=10,
        num_rollout_workers=0,
        enable_connectors=False,
    )
    .framework("torch")
    .training(
        model={"fcnet_hiddens": [64, 64]},
        lr=0.0001,
    )
    .evaluation(
        #evaluation_interval=1,
        #evaluation_num_workers=0
    )
)


algo = config.build(use_copy=True)

/home/user@AD/lab/EnergyPlus-OOEP/.venv/lib/python3.11/site-packages/ray/rllib/algorithms/algorithm.py:500: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by setting env variable PYTHONWARNINGS="ignore::DeprecationWarning"
`UnifiedLogger` will be removed in Ray 2.7.
  return UnifiedLogger(config, logdir, loggers=None)
/home/user@AD/lab/EnergyPlus-OOEP/.venv/lib/python3.11/site-packages/ray/tune/logger/unified.py:53: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by setting env variable PYTHONWARNINGS="ignore::DeprecationWarning"
The `JsonLogger interface is deprecated in favor of the `ray.tune.json.JsonLoggerCallback` interface and will be removed in Ray 2.7.
  self._loggers.append(cls(self.config, self.logdir, self.trial))
2024-04-21 01:31:04,100	WARNING unified.py:56 -- Could not instantiate JsonLogger: cannot pickle '_hashlib.HMAC' objec

Exception ignored on calling ctypes callback function: <function EventManager._core_callback_setters.<locals>.<lambda>.<locals>.<lambda> at 0x7f8a4c247f60>
Traceback (most recent call last):
  File "/home/user@AD/lab/EnergyPlus-OOEP/packages/energyplus/ooep/components/events.py", line 65, in <lambda>
    state, lambda s: self.trigger(
                     ^^^^^^^^^^^^^
  File "/home/user@AD/lab/EnergyPlus-OOEP/packages/energyplus/ooep/components/events.py", line 133, in trigger
    raise e
  File "/home/user@AD/lab/EnergyPlus-OOEP/packages/energyplus/ooep/components/events.py", line 130, in trigger
    try: super().trigger(*args, **kwargs)
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
TypeError: BaseEventManager.trigger() missing 1 required positional argument: 'event'


Train the algorithm.

In [ ]:
# force simulator to rerun to make the newly requested variables available
simulator.stop()
# start training
for _ in range(5):
    print(algo.train())

2024-04-18 06:31:10,018	WARNING deprecation.py:50 -- DeprecationWarning: `ray.rllib.execution.train_ops.multi_gpu_train_one_step` has been deprecated. This will raise an error in the future!


{'custom_metrics': {}, 'episode_media': {}, 'info': {'learner': {'default_policy': {'custom_metrics': {}, 'learner_stats': {'cur_kl_coeff': 0.20000000000000004, 'cur_lr': 0.00010000000000000003, 'total_loss': 9.938055196372412, 'policy_loss': -0.062189261382946406, 'vf_loss': 10.0, 'vf_explained_var': 0.7088255363125955, 'kl': 0.0012224227473232395, 'entropy': 1.4575255892610037, 'entropy_coeff': 0.0}, 'model': {}, 'num_grad_updates_lifetime': 465.5, 'diff_num_grad_updates_vs_sampler_policy': 464.5}}, 'num_env_steps_sampled': 4000, 'num_env_steps_trained': 4000, 'num_agent_steps_sampled': 4000, 'num_agent_steps_trained': 4000}, 'sampler_results': {'episode_reward_max': nan, 'episode_reward_min': nan, 'episode_reward_mean': nan, 'episode_len_mean': nan, 'episode_media': {}, 'episodes_this_iter': 0, 'policy_reward_min': {}, 'policy_reward_max': {}, 'policy_reward_mean': {}, 'custom_metrics': {}, 'hist_stats': {'episode_reward': [], 'episode_lengths': []}, 'sampler_perf': {}, 'num_faulty_

Evaluate the algorithm.

In [ ]:
algo.evaluate()

{'evaluation': {'sampler_results': {'episode_reward_max': 9023.0,
   'episode_reward_min': 9023.0,
   'episode_reward_mean': 9023.0,
   'episode_len_mean': 9024.0,
   'episode_media': {},
   'episodes_this_iter': 4,
   'policy_reward_min': {},
   'policy_reward_max': {},
   'policy_reward_mean': {},
   'custom_metrics': {},
   'hist_stats': {'episode_reward': [9023.0, 9023.0, 9023.0, 9023.0],
    'episode_lengths': [9024, 9024, 9024, 9024]},
   'sampler_perf': {'mean_raw_obs_processing_ms': 0.12053759098410564,
    'mean_inference_ms': 1.0440719720254743,
    'mean_action_processing_ms': 0.08953358841889589,
    'mean_env_wait_ms': 2.702368601608092,
    'mean_env_render_ms': 0.0},
   'num_faulty_episodes': 0,
   'connector_metrics': {}},
  'episode_reward_max': 9023.0,
  'episode_reward_min': 9023.0,
  'episode_reward_mean': 9023.0,
  'episode_len_mean': 9024.0,
  'episode_media': {},
  'episodes_this_iter': 4,
  'policy_reward_min': {},
  'policy_reward_max': {},
  'policy_reward_mea

Stop the simulation environment when we are done!

In [ ]:
simulator.stop()